# Unit 42. 테코레이터 사용하기

파이썬은 데코레이터라는 기능을 제공한다. 데코레이터는 장식하는 도구 정도로 설명할 수 있다.
@로 시작하는 것들이 데코레이더 이다. 즉, 함수(메서드)장식한다.

```python
class Calc:
    @staticmethod  # 데코레이터
    def add(a, b):
        print(a + b)
```

## 42.1 데코레이터 만들기

데코레이터는 함수를 수정하지 않은 상태에서 추가 기능을 구현할 때 사용한다.

In [1]:
# 함수의 시작과 끝을 출력하고 싶다면 함수 시작, 끝 부분에 print를 넣는다.
def hello():
    print("function start!")
    print("hello")
    print("function end!")

def world():
    print("function start!")
    print("world")
    print("function end!")

hello()
world()

function start!
hello
function end!
function start!
world
function end!


In [2]:
# 함수가 많아지면 매우 번거로워 진다.
# 이런 경우 데코레이터를 활용하면 편리하다.

# 함수의 시작과 끝을 출력하는 데코레이터
def trace(func):  # 호출할 함수를 매개변수로 받음
    def wrapper():  # 호출할 함수를 감싸는 함수
        print(func.__name__, "function start!")  # __name__으로 함수 이름 출력
        func()  # 매개변수로 받은 함수 호출
        print(func.__name__, "function end!")
    return wrapper  # wrapper 함수 반환

def hello():
    print("hello")

def world():
    print("world")

trace_hello = trace(hello)  # 데코레이터에 호출할 함수 넣음
trace_hello()  # 반환된 함수 호출
trace_world = trace(world)  # 데코레이터에 호출할 함수 넣음
trace_world()  # 반환된 함수 호출

hello function start!
hello
hello function end!
world function start!
world
world function end!


## 42.1.1 @로 데코레이터 사용하기

좀 더 같편하게 @로 데코레이터를 사용한다.

```python
@데코레이터
def 함수이름():
    코드
```

In [3]:
def trace(func):
    def wrapper():
        print(func.__name__, "function start!")
        func()
        print(func.__name__, "function end!")
    return wrapper

@trace  # @데코레이터
def hello():
    print("hello")

@trace
def world():
    print("world")

hello()
world()

hello function start!
hello
hello function end!
world function start!
world
world function end!


> 참고: 데코레이터를 여러개 지정하기

함수에는 데코레이터를 여러개 지정할 수 있다. 데코레이터가 실행되는 순서는 위에서 아래 순이다.

```python
@데코레이터1
@데코레이터2
def 함수이름():
    코드

In [6]:
def decorator1(func):
    def wrapper():
        print("decorator1")
        func()
    return wrapper

def decorator2(func):
    def wrapper():
        print("decorator2")
        func()
    return wrapper

# 데코레이터를 여러개 지정
@decorator1
@decorator2
def hello():
    print("hello")

hello()
print()

def world():
    print("world")

# @를 사용하지 않았을 때
decorator_hello = decorator1(decorator2(world))
decorator_hello()

decorator1
decorator2
hello

decorator1
decorator2
world


## 42.2. 매개변수와 반환값을 처리하는 데코레이터 만들기

매개변수와 반환값을 처리하는 데코레이터 만드는 방법

In [7]:
def trace(func):
    def wrapper(a, b):  # 호출 함수 add(a, b)의 매개변수와 똑같이 지정
        r = func(a, b)  # func에 매개변수 a, b를 넣어서 호출하고 반환값을 변수에 저장
        print("{0}(a={1}, b={2}) -> {3}".format(func.__name__, a, b, r))  # 매개변수와 반환값 출력
        return r  # func의 반환값을 반환
    return wrapper

@trace  # @데코레이터
def add(a, b):
    return a + b

print(add(10, 20))

add(a=10, b=20) -> 30
30


## 42.2.1 가변 인수 함수 데코레이터

매개변수(인수)가 고정되지 않은 함수는 어떻게 처리할까?
이 때는 wrapper 함수를 가변 인수로 만들면 된다.

In [9]:
def trace(func):
    def wrapper(*args, **kwargs):  # 가변인수
        r = func(*args, **kwargs)
        print("{0}(args={1}, kwargs={2} -> {3}".format(func.__name__, args, kwargs, r))
        return r
    return wrapper

@trace
def get_max(*args):
    return max(args)

@trace
def get_min(**kwargs):
    return min(kwargs.values())

print(get_max(10, 20))
print(get_min(x=10, y=20, z=30))

get_max(args=(10, 20), kwargs={} -> 20
20
get_min(args=(), kwargs={'x': 10, 'y': 20, 'z': 30} -> 10
10


> 참고: 메서드에 데코레이터 사용하기

클래스를 만들면서 메서드에 데코레이터를 사용할 때는 self를 주의해야 한다.
인스턴스 메서드는 항상 self를 받으므로 데코레이터를 만들 때도 wrapper 함수의 첫 번째 매개변수는 self로 지정해야 한다.(클래스 메서드는 cls)
마찬가지로 func를 호출할 때도 self와 매개변수를 그대로 넣어야 한다.

In [11]:
def trace(func):
    def wrapper(self, a, b):  # 호출할 함수가 인스턴스 메서드이므로 첫 번째 매개변수는 self
        r = func(self, a, b)  # self와 매개변수를 그대로 넣어줌
        print("{0}(a={1}, b={2}) -> {3}".format(func.__name__, a, b, r))
        return r
    return wrapper

class Calc:
    @trace
    def add(self, a, b):
        return a + b

c = Calc()
print(c.add(10, 20))

add(a=10, b=20) -> 30
30


## 42.3. 매개변수가 있는 데코레이터 만들기

매개변수가 있는 데코레이터는 값을 지정해서 동작을 바꿀 수 있다.

In [12]:
# 함수의 반환값이 특정 수의 배수인지 확인하는 데코레이터

def is_multiple(x):  # 데코레이터가 사용할 매개변수 지정
    def real_decorator(func):  # 호출할 함수를 매개변수로 받음
        def wrapper(a, b):  # 호출할 함수의 매개변수와 똑같이 지정
            r = func(a, b)
            if r % x == 0:  # func의 반환값이 x의 배수인지 확인
                print("{0}의 반환값은 {1}의 배수입니다.".format(func.__name__, x))
            else:
                print("{0}의 반환값은 {1}의 배수가 아닙니다.".format(func.__name__, x))
            return r
        return wrapper
    return real_decorator

@is_multiple(3)  # @데코레이터(인수)
def add(a, b):
    return a + b

print(add(10, 20))
print(add(2, 5))

add의 반환값은 3의 배수입니다.
30
add의 반환값은 3의 배수가 아닙니다.
7


> 참고: 매개변수가 있는 데코레이터 여러개 지정하기

In [16]:
@is_multiple(3)
@is_multiple(7)
def add(a, b):
    return a + b

print(add(10, 20))

def add2(a, b):
    return a + b

# @를 사용하지 않을 때
decorated_add = is_multiple(3)(is_multiple(7)(add2))
print(decorated_add(10, 20))

add의 반환값은 7의 배수가 아닙니다.
wrapper의 반환값은 3의 배수입니다.
30
add2의 반환값은 7의 배수가 아닙니다.
wrapper의 반환값은 3의 배수입니다.
30


> 참고: 원래 함수 이름이 안 나온다면?

데코레이터를 여려 개 사용하면 데코레이터에서 반환된 wrapper 함수가 다른 데코레이터로 들어간다. 따라서 함수의 __name__을 출력해보면 wrapper가 나오는데 원래 함수 이름을 출력하고 싶다면 functools 모듈의 wraps 데코레이터를 사용해야 한다.

In [17]:
import functools

def is_multiple(x):
    def real_decorator(func):
        @functools.wraps(func)  # functools.wraps에 func를 넣은 뒤 wrapper 함수 위에 지정
        def wrapper(a, b):
            r = func(a, b)
            if r % x == 0:
                print("{0}의 반환값은 {1}의 배수입니다.".format(func.__name__, x))
            else:
                print("{0}의 반환값은 {1}의 배수가 아닙니다.".format(func.__name__, x))
            return r
        return wrapper
    return real_decorator

@is_multiple(3)
@is_multiple(7)
def add(a, b):
    return a + b

print(add(10, 20))

# @functools.wraps는 원래 함수의 정보를 유지시켜준다. 디버깅 할 때 유용하므로 데코레이터를 만들 때는 @functools.wraps를 사용하는 것이 좋다.

add의 반환값은 7의 배수가 아닙니다.
add의 반환값은 3의 배수입니다.
30


## 42.4 클래스로 데코레이터 만들기

클래스를 활용할 때는 인스턴스 함수처럼 호출하게 해주는 __call__메서드를 구현해야 한다.

In [18]:
class Trace:
    def __init__(self, func):  # 호출할 함수를 인스턴스 초기값으로 받음
        self.func = func
    
    def __call__(self):
        print(self.func.__name__, "func start!")
        self.func()
        print(self.func.__name__, "func end!")


@Trace
def hello():
    print("hello")

hello()

hello func start!
hello
hello func end!


In [19]:
# @데코레이터를 지정하지 않는 방법
def world():
    print("world")

trace_world = Trace(world)
trace_world()  # 인스턴스를 호출, __call__ 메서드가 호출됨

world func start!
world
world func end!


## 42.5 클래스로 매개변수와 반환값을 처리하는 데코레이터 만들기

In [20]:
class Trace:
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args, **kwargs):
        r = self.func(*args, **kwargs)
        print("{0}(args={1}, kwargs={2}) -> {3}".format(self.func.__name__, args, kwargs, r))
        return r


@Trace
def add(a, b):
    return a + b

print(add(10, 20))
print(add(a=10, b=20))

add(args=(10, 20), kwargs={}) -> 30
30
add(args=(), kwargs={'a': 10, 'b': 20}) -> 30
30


### 42.5.1 클래스로 매개변수가 있는 데코레이터 만들기

In [21]:
# 함수의 반환값이 특정 수의 배수인지 확인
class IsMultiple:
    def __init__(self, x):  # 데코레이터가 사용할 매개변수를 초기값으로 받음
        self.x = x
    
    def __call__(self, func):  # 호출할 함수를 매개변수로 받음
        def wrapper(a, b):  # 호출할 함수의 매개변수와 똑같이 지정
            r = func(a, b)
            if r % self.x == 0:
                print("{0}의 반환값은 {1}의 배수입니다.".format(func.__name__, self.x))
            else:
                print("{0}의 반환값은 {1}의 배수가 아닙니다.".format(func.__name__, self.x))
            return r
        return wrapper

@IsMultiple(3)
def add(a, b):
    return a + b

print(add(10, 20))
print(add(2, 5))

add의 반환값은 3의 배수입니다.
30
add의 반환값은 3의 배수가 아닙니다.
7
